In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import OpenEXR

In [ ]:
def crop_exr(input_path, output_path, x, y, w, h):
  image = OpenEXR.File(input_path)
  header = image.header()
  wind = (np.array([0, 0], dtype=np.int32),
          np.array([w-1, h-1], dtype=np.int32))
  header['dataWindow'] = wind
  header['displayWindow'] = wind

  def crop_inner(c):
    c = c[y:y+h, x:x+w]
    return np.copy(c) # wouldn't work without copy, OpenEXR bug

  channels = {}
  for name, chan in image.channels().items():
    channels[name] = crop_inner(chan.pixels)

  with OpenEXR.File(header, channels) as outfile:
    outfile.write(output_path)

  #again = OpenEXR.File(output_path)
  #for name, chan in again.channels().items():
  #  display(chan)
  #  px.imshow(chan.pixels).show()

In [ ]:
crop_exr('../exr/cam7.exr',
         '../exr/cam7-cropped.exr',
         0, 1000, 600, 400)

In [ ]:
img = np.zeros((50,48))

for i in range(img.shape[0]):
  for j in range(img.shape[1]):
    ii = i
    jj = j
    if (ii + jj) % 2 == 0:
      img[i][j] = 20
    else:
      img[i][j] = - ii - jj

fig = px.imshow(img)
fig.update_layout(height = 800)
fig.show()


In [ ]:
# ×

names = [
  "1920×1080",
  "500×500",
  "1700×2200",
  "1875×2500",
  "375×375",
]

fig = go.Figure(data=[
  go.Bar(name = 'без unroll', x=names, y=[4.030, 3.595, 2.538, 2.895, 3.015,]),
  go.Bar(name = 'с unroll',   x=names, y=[7.183, 6.747, 5.177, 5.754, 6.034,]),
])
fig.update_layout(yaxis_title='Производительность, Мпиксель/с')
fig.show()